In [2]:
%cd d:\Laptop_Projects\LLMEval-1

d:\Laptop_Projects\LLMEval-1


In [3]:
from collections import Counter
import numpy as np
from summac.model_guardrails import NERInaccuracyPenalty
import nltk
import spacy
import json
import pandas as pd
import string
import tfidf_matcher as tm
from fuzzywuzzy import fuzz
from server.metrics.faithfulness_helper import FaithfulnessHelper,get_quartiles_faithfulness
from server.metrics.stylistic import FaithfulnessEvaluator

data = json.load(open(r'preprocess\experiments\data\pairwise_evaluation_w_embeddings.json'))

In [4]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

Load data, all class objects and meta data

In [14]:
data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_test.json'))
meta_data = json.load(open(r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\meta_data.json'))
df = pd.DataFrame(data)
fh = FaithfulnessHelper()
md = meta_data["faithfulness"]["ranges"]
ranges = []
for range in md:
    ranges.append([range[0],range[1]])
fe = FaithfulnessEvaluator(ranges) #Pass empty string if you dont have Faithfulness_test.json

New Instance example

In [5]:
article_text = 'Polls close at 11 p.m. Eastern time. In the 2022 state primaries, first results were reported 11 minutes later, and the last update of the night was at 6:10 a.m. Eastern time with 48 percent of votes reported.All active registered voters were mailed a ballot, which must be postmarked by Election Day and received by March 12. Early in-person voting was also available.California is one of five states on Tuesday holding primaries for the presidential race as well as its state primary. In state and congressional primaries, candidates from all parties are listed on one ballot, and the top two vote-getters advance to the general election. In the presidential primary, Republican voters were required to be registered with their party to vote for its nominee, but Democrats and unaffiliated voters could participate in the Democratic election.'
summ ='In the 2022 California state primaries, polls closed at 11 p.m. Eastern time. First results were reported 11 minutes later, and the last update of the night was at 6:10 a.m. Eastern time with 48 percent of votes reported. California held primaries for both the presidential race and its state primary, using a top-two vote-getter system for state and congressional primaries.'
fe.default(article_text,summ)

(0.6666666666666666, 'good')

Calculate scores for whole dataset

In [15]:
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data)))
    article=datum['article_text']
    LLM=datum['text-davinci-002_summary']
    datum['faithfullness'], datum['faithfullness_binary'] = fe.default(article,LLM) # Remove 'datum['faithfullness_binary']' if you dont have Faithfulness_test.json 
    print("score",datum['faithfullness'])

0/599
score 0.5
1/599
score 0.8
2/599
score 0.7777777777777778
3/599
score 0.6666666666666666
4/599
score 0.2
5/599
score 0.75
6/599
score 0.6
7/599
score 0.0
8/599


KeyboardInterrupt: 

In [7]:
save_json(data,r'D:\Laptop_Projects\LLMEval-1\preprocess\experiments\data\Faithfulness_binary.json')